# Small Task Packing

This notebook demonstrates the workload behaviour for small task packing

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Comment the follwing line to disable devlib debugging statements
# logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
# Generate plots inline
%pylab inline

import json
import os
import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace
from trace_analysis import TraceAnalysis

# Support to configure and run RTApp based workloads
from wlgen import RTA

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [3]:
# Setup a target configuration
my_target_conf = {
    
    # Define the kind of target platform to use for the experiments
    "platform"    : 'android',  # Linux system, valid other options are:
                              # android - access via ADB
                              # linux   - access via SSH
                              # host    - direct access
    
    # Preload settings for a specific target
    "board"       : 'juno',   # load JUNO specific settings, e.g.
                              # - HWMON based energy sampling
                              # - Juno energy model
                              # valid options are:
                              # - juno  - JUNO Development Board
                              # - tc2   - TC2 Development Board
                              # - oak   - Mediatek MT63xx based target

    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Account to access the remote target
    "host"        : '192.168.1.100',
    "username"    : 'root',
    "password"    : '',

    # Comment the following line to force rt-app calibration on your target
    "rtapp-calib" : {
        '0': 361, '1': 138, '2': 138, '3': 352, '4': 360, '5': 353
    }

}

# Setup the required Test Environment supports
my_tests_conf = {
    
    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    #"tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_switch",
            'dequeue_task_fair', 
            'enqueue_task_fair', 
            'set_next_entity',

                    
            "cpu_capacity",
            
            #/sys/kernel/debug/tracing/events/sched
            "sched_blocked_reason",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_contrib_scale_f",
            "sched_cpu_hotplug",
            "sched_energy_diff",
            "sched_kthread_stop",
            "sched_kthread_stop_ret",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_migrate_task",
            "sched_move_numa",
            "sched_pi_setprio",
            "sched_process_exec",
            "sched_process_exit",
            "sched_process_fork",
            "sched_process_free",
            "sched_process_wait",
            "sched_stat_blocked",
            "sched_stat_iowait",
            "sched_stat_runtime",
            "sched_stat_sleep",
            "sched_stat_wait",
            "sched_stick_numa",
            "sched_swap_numa",
            "sched_switch",
            "sched_tune_boostgroup_update",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_filter",
            "sched_wait_task",
            "sched_wake_idle_without_ipi",
            "sched_wakeup",
            "sched_wakeup_new",
            
            
         ],
         "buffsize" : 10240
    },
    "results_dir" : "lisa_small_task_packing_2",
}

In [4]:
# Support to access the remote target
import devlib
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

10:24:52  INFO    :         Target - Using base path: /data/lisa_fork/lisa
10:24:52  INFO    :         Target - Loading custom (inline) target configuration
10:24:52  INFO    :         Target - Loading custom (inline) test configuration
10:24:52  INFO    :         Target - External tools using:
10:24:52  INFO    :         Target -    ANDROID_HOME: /home/zhifei/zyang/android-sdk-linux
10:24:52  INFO    :         Target -    CATAPULT_HOME: /data/lisa_fork/lisa/tools/catapult
10:24:52  INFO    :         Target - Devlib modules to load: ['bl', 'hwmon', 'cpufreq']
10:24:52  INFO    :         Target - Connecting Android target [192.168.1.100:5555]
10:24:52  INFO    :         Target - Connection settings:
10:24:52  INFO    :         Target -    {'device': '192.168.1.100:5555'}
10:24:54  INFO    :         Target - Initializing target workdir:
10:24:54  INFO    :         Target -    /data/local/tmp/devlib-target
10:24:58  INFO    :         Target - Topology:
10:24:58  INFO    :         Target -

# Workload configuration

In [5]:
# Support to configure and run RTApp based workloads
from wlgen import RTA, Periodic, Ramp

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp = RTA(target, 'simple', calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(
    # 1. generate a "profile based" set of tasks
    kind='profile',
    
    # 2. define the "profile" of each task
    params={
        'stp_0': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration
        ).get(),
        'stp_1': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration
            delay_s=0.01
        ).get(),
        'stp_2': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration    
            delay_s=0.02
        ).get(),
        'stp_3': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration    
            delay_s=0.035
        ).get(),
        'stp_4': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration
            delay_s=0.05
        ).get(),
        'stp_5': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration    
            delay_s=0.065
        ).get(),
        'stp_6': Periodic(
            period_ms=100,         # period
            duty_cycle_pct=2,     # duty cycle
            duration_s=10,          # duration    
            delay_s=0.08
        ).get(),

    },    
    
    # 4. use this folder for task logfiles
    run_dir=target.working_directory
    
);

10:25:42  INFO    :          WlGen - Setup new workload simple
10:25:42  DEBUG   :          WlGen - Setup step [postrun] callback to [__postrun] function
10:25:42  DEBUG   :          WlGen - Configuring a profile-based workload...
10:25:42  DEBUG   :          RTApp - ref on cpu: 1
10:25:42  INFO    :          RTApp - Workload duration defined by longest task
10:25:42  INFO    :          RTApp - Default policy: SCHED_OTHER
10:25:42  INFO    :          RTApp - ------------------------
10:25:42  INFO    :          RTApp - task [stp_0], sched: using default policy
10:25:42  INFO    :          RTApp -  | calibration CPU: 1
10:25:42  INFO    :          RTApp -  | loops count: 1
10:25:42  INFO    :          RTApp - + phase_000001: duration 10.000000 [s] (100 loops)
10:25:42  INFO    :          RTApp - |  period   100000 [us], duty_cycle   2 %
10:25:42  INFO    :          RTApp - |  run_time   2000 [us], sleep_time  98000 [us]
10:25:42  INFO    :          RTApp - ------------------------
10:25

# Workload execution

In [6]:
# Set Sched governor
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('sched')

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU max CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU max CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq")
             )

10:29:42  INFO    : Target ABI: arm64, CPus: ['A53', 'A57', 'A57', 'A53', 'A53', 'A53']
10:29:42  DEBUG   : adb -s 192.168.1.100:5555 shell "echo '/data/local/tmp/bin/shutils cpufreq_set_all_governors sched' | su"
10:29:42  DEBUG   : adb -s 192.168.1.100:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'"
10:29:42  INFO    : Target current governor: sched
10:29:42  DEBUG   : adb -s 192.168.1.100:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq'"
10:29:42  DEBUG   : adb -s 192.168.1.100:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq'"
10:29:42  INFO    : Target big CPU max CPUfreq:850000
		    Target big CPU current CPUfreq: 850000
10:29:42  DEBUG   : adb -s 192.168.1.100:5555 shell "cat '/sys/devices/system/cpu/cpu1/cpufreq/scaling_max_freq'"
10:29:42  DEBUG   : adb -s 192.168.1.100:5555 shell "cat '/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq'"
10:29:42  INFO    : Target LITTLE CPU max CPUfreq:1100000 
		    T

In [7]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
(nrg, nrg_file) = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

10:29:46  INFO    : #### Setup FTrace
10:29:46  DEBUG   : adb -s 192.168.1.100:5555 shell "echo 'echo 10240 > '\\''/sys/kernel/debug/tracing/buffer_size_kb'\\''' | su"
10:29:46  DEBUG   : adb -s 192.168.1.100:5555 shell "cat '/sys/kernel/debug/tracing/buffer_size_kb'"
10:29:46  DEBUG   : adb -s 192.168.1.100:5555 shell "echo '/data/local/tmp/bin/trace-cmd reset' | su"
10:29:47  DEBUG   : adb -s 192.168.1.100:5555 shell "echo '/data/local/tmp/bin/trace-cmd start -e cpu_frequency -e sched_load_avg_cpu -e sched_load_avg_task -e sched_switch -e cpu_capacity -e sched_blocked_reason -e sched_boost_cpu -e sched_boost_task -e sched_contrib_scale_f -e sched_cpu_hotplug -e sched_energy_diff -e sched_kthread_stop -e sched_kthread_stop_ret -e sched_load_avg_cpu -e sched_load_avg_task -e sched_migrate_task -e sched_move_numa -e sched_pi_setprio -e sched_process_exec -e sched_process_exit -e sched_process_fork -e sched_process_free -e sched_process_wait -e sched_stat_blocked -e sched_stat_iowait -e 

# Collected results

In [8]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
#!ls -la {te.res_dir}

10:30:04  INFO    : Content of the output folder /data/lisa_fork/lisa/results/lisa_small_task_packing_2


In [9]:
# Inspect the JSON file used to run the application
with open('{}/simple_00.json'.format(te.res_dir), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
#print json.dumps(rtapp_json, indent=4, sort_keys=True)

10:30:11  INFO    : Generated RTApp JSON file:


In [10]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_file)
print json.dumps(nrg, indent=4, sort_keys=True)

10:30:16  INFO    : Energy: /data/lisa_fork/lisa/results/lisa_small_task_packing_2/energy.json


{
    "LITTLE": 1.1182609999999897, 
    "big": 0.8741820000000047
}


In [11]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
#print json.dumps(plt, indent=4, sort_keys=True)

10:30:17  INFO    : Platform description: /data/lisa_fork/lisa/results/lisa_small_task_packing_2/platform.json


# Trace inspection

In [12]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

# New Task Behavior

In [13]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['stp_0',   "stp_1", "stp_2",
                                             "stp_3", "stp_4", "stp_5", "stp_6"
                                            ])

10:31:06  DEBUG   : Loading [sched] events from trace in [/data/lisa_fork/lisa/results/lisa_small_task_packing_2]...
10:31:06  DEBUG   : Parsing events: ['cpu_frequency', 'sched_load_avg_cpu', 'sched_load_avg_task', 'sched_switch', 'dequeue_task_fair', 'enqueue_task_fair', 'set_next_entity', 'cpu_capacity', 'sched_blocked_reason', 'sched_boost_cpu', 'sched_boost_task', 'sched_contrib_scale_f', 'sched_cpu_hotplug', 'sched_energy_diff', 'sched_kthread_stop', 'sched_kthread_stop_ret', 'sched_load_avg_cpu', 'sched_load_avg_task', 'sched_migrate_task', 'sched_move_numa', 'sched_pi_setprio', 'sched_process_exec', 'sched_process_exit', 'sched_process_fork', 'sched_process_free', 'sched_process_wait', 'sched_stat_blocked', 'sched_stat_iowait', 'sched_stat_runtime', 'sched_stat_sleep', 'sched_stat_wait', 'sched_stick_numa', 'sched_swap_numa', 'sched_switch', 'sched_tune_boostgroup_update', 'sched_tune_config', 'sched_tune_tasks_update', 'sched_tune_filter', 'sched_wait_task', 'sched_wake_idle_w